<a href="https://colab.research.google.com/github/Salma-Kassem/optmization_techniques/blob/main/LogisticRergression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn.datasets import (make_classification , load_breast_cancer,make_moons)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, accuracy_score
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
class LogisticRegressionGD:
    def __init__(self, learning_rate=0.01, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = 0

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def loss(self, y, y_pred):
        m = len(y)
        return -(1/m) * np.sum(y * np.log(y_pred + 1e-15) + (1 - y) * np.log(1 - y_pred + 1e-15))  # +eps for stability

    def fit(self, X, y):
        m, n = X.shape
        self.weights = np.zeros(n)

        for epoch in range(self.epochs):
            z = np.dot(X, self.weights) + self.bias
            y_pred = self.sigmoid(z)

            current_loss = self.loss(y, y_pred)

            dw = (1/m) * np.dot(X.T, (y_pred - y))
            db = (1/m) * np.sum(y_pred - y)

            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {current_loss:.4f}")

    def predict(self, X):
        z = np.dot(X, self.weights) + self.bias
        return (self.sigmoid(z) >= 0.5).astype(int)

data = load_breast_cancer()
X, y = data.data, data.target
X_class, y_class = make_classification(n_samples=500, n_features=10, n_classes=2, random_state=0)
# Standardize the features (important for gradient descent)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegressionGD(learning_rate=0.01, epochs=1000)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
loss = log_loss(y_test, y_pred)
print(f"Log Loss (binary cross-entropy): {loss:.4f}")
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
#Accuracy = (TP + TN) / (TP + TN + FP + FN) Precision = TP / (TP + FP) Recall = TP / (TP + FN) F1 Score = 2 * (Precision * Recall) / (Precision + Recall)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy:  {acc:.2f}")
print(f"Precision: {prec:.2f}")
print(f"Recall:    {rec:.2f}")
print(f"F1 Score:  {f1:.2f}")
class LinearRegression:
     def __init__(self):
        self.weights = None
        self.bias = None
     def fit(self, X, y):
        X_b = np.c_[np.ones((X.shape[0], 1)), X]
        # Normal Equation: θ = (XᵀX)⁻¹Xᵀy instead of gradient descent (itterative method) for optimal weights and bias [no loop no learning rate ][don't work well with large data ,numerically tricky]
        theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

        # Save bias and weights
        self.bias = theta_best[0]
        self.weights = theta_best[1:]
     def predict(self, X):
        return np.dot(X, self.weights) + self.bias
     def score(self, X, y):
         y_pred = self.predict(X)
         ss_total = np.sum((y-np.mean(y)) ** 2)
         ss_res = np.sum((y - y_pred) ** 2)
         r2 = 1 - (ss_res / ss_total)
         return r2

Epoch 0, Loss: 0.6931
Epoch 100, Loss: 0.2522
Epoch 200, Loss: 0.1897
Epoch 300, Loss: 0.1615
Epoch 400, Loss: 0.1448
Epoch 500, Loss: 0.1336
Epoch 600, Loss: 0.1253
Epoch 700, Loss: 0.1190
Epoch 800, Loss: 0.1139
Epoch 900, Loss: 0.1097
Log Loss (binary cross-entropy): 0.3162
Confusion Matrix:
[[42  1]
 [ 0 71]]
Accuracy:  0.99
Precision: 0.99
Recall:    1.00
F1 Score:  0.99


In [ ]:
import numpy as np
from sklearn.datasets import (make_classification , load_breast_cancer,make_moons)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

class LogisticRegressionSGD:
    def __init__(self, learning_rate=0.01, epochs=1000):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.weights = None
        self.bias = 0

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def loss(self, y, y_pred):
        m = len(y)
        return -(1/m) * np.sum(
            y * np.log(y_pred + 1e-15) +
            (1 - y) * np.log(1 - y_pred + 1e-15)
        )  # Adding a small constant for numerical stability

    def fit(self, X, y):
        m, n = X.shape
        self.weights = np.zeros(n)
        self.bias = 0

        for epoch in range(self.epochs):
            # Shuffle the training data for stochastic gradient descent
            indices = np.random.permutation(m)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            for i in range(m):  # Loop through each training sample
                xi = X_shuffled[i:i+1]  # Extract one sample
                yi = y_shuffled[i:i+1]  # Extract the corresponding label

                # Compute prediction for the current sample
                z = np.dot(xi, self.weights) + self.bias
                y_pred = self.sigmoid(z)

                # Compute gradients for the current sample
                dw = xi.T * (y_pred - yi)  # Gradient for weights
                db = y_pred - yi  # Gradient for bias

                # Flatten dw to make sure it matches the shape of self.weights
                self.weights -= self.learning_rate * dw.flatten()
                self.bias -= self.learning_rate * db

            # Optionally print the loss every 100 epochs
            if epoch % 100 == 0:
                y_pred = self.sigmoid(np.dot(X, self.weights) + self.bias)
                current_loss = self.loss(y, y_pred)
                print(f"Epoch {epoch}, Loss: {current_loss:.4f}")

    def predict(self, X):
        z = np.dot(X, self.weights) + self.bias
        return (self.sigmoid(z) >= 0.5).astype(int)

# Load and preprocess data
data = load_breast_cancer()
X, y = data.data, data.target
X_class, y_class = make_classification(n_samples=500, n_features=10, n_classes=2, random_state=0)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model using Stochastic Gradient Descent
model_sgd = LogisticRegressionSGD(learning_rate=0.01, epochs=1000)
model_sgd.fit(X_train, y_train)

# Predict on the test set
y_pred = model_sgd.predict(X_test)

# Evaluation
loss = log_loss(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
#Accuracy = (TP + TN) / (TP + TN + FP + FN) Precision = TP / (TP + FP) Recall = TP / (TP + FN) F1 Score = 2 * (Precision * Recall) / (Precision + Recall)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy:  {acc:.2f}")
print(f"Precision: {prec:.2f}")
print(f"Recall:    {rec:.2f}")
print(f"F1 Score:  {f1:.2f}")


Epoch 0, Loss: 0.1375
Epoch 100, Loss: 0.0481
Epoch 200, Loss: 0.0437
Epoch 300, Loss: 0.0411
Epoch 400, Loss: 0.0392
Epoch 500, Loss: 0.0377
Epoch 600, Loss: 0.0365
Epoch 700, Loss: 0.0353
Epoch 800, Loss: 0.0344
Epoch 900, Loss: 0.0335
Confusion Matrix:
[[42  1]
 [ 2 69]]
Accuracy:  0.97
Precision: 0.99
Recall:    0.97
F1 Score:  0.98


In [ ]:
import numpy as np
from sklearn.datasets import (make_classification , load_breast_cancer,make_moons)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

class LogisticRegressionMiniBatchGD:
    def __init__(self, learning_rate=0.01, epochs=1000, batch_size=32):
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        self.weights = None
        self.bias = 0

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def loss(self, y, y_pred):
        m = len(y)
        return -(1/m) * np.sum(
            y * np.log(y_pred + 1e-15) +
            (1 - y) * np.log(1 - y_pred + 1e-15)
        )  # Adding a small constant for numerical stability

    def fit(self, X, y):
        m, n = X.shape
        self.weights = np.zeros(n)
        self.bias = 0

        for epoch in range(self.epochs):
            # Shuffle the training data for mini-batch gradient descent
            indices = np.random.permutation(m)
            X_shuffled = X[indices]
            y_shuffled = y[indices]

            # Process data in mini-batches
            for i in range(0, m, self.batch_size):
                # Create mini-batch
                X_batch = X_shuffled[i:i + self.batch_size]
                y_batch = y_shuffled[i:i + self.batch_size]

                # Compute predictions for the mini-batch
                z = np.dot(X_batch, self.weights) + self.bias
                y_pred = self.sigmoid(z)

                # Compute gradients for the mini-batch
                dw = np.dot(X_batch.T, (y_pred - y_batch)) / len(y_batch)  # Gradient for weights
                db = np.sum(y_pred - y_batch) / len(y_batch)  # Gradient for bias

                # Update weights and bias using the gradients
                self.weights -= self.learning_rate * dw
                self.bias -= self.learning_rate * db

            # Optionally print the loss every 100 epochs
            if epoch % 100 == 0:
                y_pred = self.sigmoid(np.dot(X, self.weights) + self.bias)
                current_loss = self.loss(y, y_pred)
                print(f"Epoch {epoch}, Loss: {current_loss:.4f}")

    def predict(self, X):
        z = np.dot(X, self.weights) + self.bias
        return (self.sigmoid(z) >= 0.5).astype(int)

# Load and preprocess data
data = load_breast_cancer()
X, y = data.data, data.target
X_class, y_class = make_classification(n_samples=500, n_features=10, n_classes=2, random_state=0)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train model using Mini-Batch Gradient Descent
model_mbgd = LogisticRegressionMiniBatchGD(learning_rate=0.01, epochs=1000, batch_size=32)
model_mbgd.fit(X_train, y_train)

# Predict on the test set
y_pred = model_mbgd.predict(X_test)

# Evaluation
loss = log_loss(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)
#Accuracy = (TP + TN) / (TP + TN + FP + FN) Precision = TP / (TP + FP) Recall = TP / (TP + FN) F1 Score = 2 * (Precision * Recall) / (Precision + Recall)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy:  {acc:.2f}")
print(f"Precision: {prec:.2f}")
print(f"Recall:    {rec:.2f}")
print(f"F1 Score:  {f1:.2f}")

Epoch 0, Loss: 0.4955
Epoch 100, Loss: 0.0939
Epoch 200, Loss: 0.0787
Epoch 300, Loss: 0.0718
Epoch 400, Loss: 0.0675
Epoch 500, Loss: 0.0645
Epoch 600, Loss: 0.0623
Epoch 700, Loss: 0.0606
Epoch 800, Loss: 0.0591
Epoch 900, Loss: 0.0579
Confusion Matrix:
[[42  1]
 [ 0 71]]
Accuracy:  0.99
Precision: 0.99
Recall:    1.00
F1 Score:  0.99
